In [7]:
import pandas as pd
from pathlib import Path

In [26]:
import seaborn as sns
import tabulate

In [49]:
# PDF metadata + additional PDF metadata files, in our case my supplemental OCR of PDF BBK documents
PDF_CSV = Path("../data/FULL_CLI_2023-09-27T18:08.csv")
PDF_CSV_ADDITIONAL = [Path("../data/remaining_BB_monthly_2023-10-16T18:43.csv")]

# Spacy/lang.det metadata
TXT_CSV = Path("../data/FULL_CLI_LD_multi_new.csv")

In [50]:
pdf_csv_files = PDF_CSV_ADDITIONAL + [PDF_CSV]
pdf_csv_files

[PosixPath('../data/remaining_BB_monthly_2023-10-16T18:43.csv'),
 PosixPath('../data/FULL_CLI_2023-09-27T18:08.csv')]

In [51]:
# df_pdf/df_spacy reversed here but doesn't matter
df_pdf = pd.read_csv(TXT_CSV)
#df_spacy = pd.concat([pd.read_csv(x) for x in pdf_csv_files])
df_spacy = pd.read_csv(PDF_CSV)

len(df_pdf),len(df_spacy)

(12465, 13661)

In [52]:
df_pdf.head()

,txt_fn,num_chars,num_tokens,num_sentences,langs_raw,langs_small
0,Final_terms/DE000HLB3YZ6.pdf.txt,79473,12511,482,"Counter({'GERMAN': 0.9659390741184937, 'PORTUG...","{'GERMAN': 0.97020558946354, 'PORTUGUESE': 0.0..."
1,Final_terms/DE000A3MQUY1.pdf.txt,85077,15753,909,"Counter({'GERMAN': 0.48832783557472725, 'ENGLI...","{'GERMAN': 0.501661779081134, 'ENGLISH': 0.453..."
2,Final_terms/DE000DK0SX17.pdf.txt,141722,22583,871,"Counter({'GERMAN': 0.9567252494921841, 'ITALIA...","{'GERMAN': 0.9628905381983023, 'ITALIAN': 0.02..."
3,Final_terms/DE000LB1M1A2.pdf.txt,128745,21028,549,"Counter({'GERMAN': 0.9849795607947524, 'ENGLIS...","{'GERMAN': 0.9886450381679389, 'ENGLISH': 0.00..."
4,Final_terms/DE000A30VAY8.pdf.txt,1884,395,20,"Counter({'GERMAN': 0.8905852417302799, 'SPANIS...","{'GERMAN': 0.8905852417302799, 'SPANISH': 0.05..."


In [53]:
df_spacy.head()

,doc_path,num_pages,metadata,is_empty,txt_file,source_fn,txt_fn,collection,error,exception_text
0,/home/sh/hsa/papers/dataset/extracted/Final_te...,19.0,"{'format': 'PDF 1.4', 'title': 'EA', 'author':...",False,/tmp/FULL_CLI/Final_terms/DE000HLB7C13.pdf.txt,Final_terms/Country_of_origin_DE/lang_DE_EN/DE...,FULL_CLI/Final_terms/DE000HLB7C13.pdf.txt,Final_terms,NaN,NaN
1,/home/sh/hsa/papers/dataset/extracted/Final_te...,14.0,"{'format': 'PDF 1.6', 'title': '', 'author': '...",False,/tmp/FULL_CLI/Final_terms/DE000NWB2ME7.pdf.txt,Final_terms/Country_of_origin_DE/lang_DE_EN/DE...,FULL_CLI/Final_terms/DE000NWB2ME7.pdf.txt,Final_terms,NaN,NaN
2,/home/sh/hsa/papers/dataset/extracted/Final_te...,25.0,"{'format': 'PDF 1.5', 'title': '', 'author': '...",False,/tmp/FULL_CLI/Final_terms/DE000DFK0TS8.pdf.txt,Final_terms/Country_of_origin_DE/lang_DE_EN/DE...,FULL_CLI/Final_terms/DE000DFK0TS8.pdf.txt,Final_terms,NaN,NaN
3,/home/sh/hsa/papers/dataset/extracted/Final_te...,15.0,"{'format': 'PDF 1.4', 'title': 'Dokumentation'...",False,/tmp/FULL_CLI/Final_terms/DE000LB2CZ54.pdf.txt,Final_terms/Country_of_origin_DE/lang_DE_EN/DE...,FULL_CLI/Final_terms/DE000LB2CZ54.pdf.txt,Final_terms,NaN,NaN
4,/home/sh/hsa/papers/dataset/extracted/Final_te...,25.0,"{'format': 'PDF 1.5', 'title': '', 'author': '...",False,/tmp/FULL_CLI/Final_terms/DE000BLB80W4.pdf.txt,Final_terms/Country_of_origin_DE/lang_DE_EN/DE...,FULL_CLI/Final_terms/DE000BLB80W4.pdf.txt,Final_terms,NaN,NaN


In [54]:
# Remove forgotten artifact from txt_fn path - fixed the cause in the .py file
df_spacy.txt_fn = df_spacy.txt_fn.str.replace("FULL_CLI/","")

In [55]:
# Merge both on the txt_fn column, leading to the complete list of all files, empty/error or not
df = df_spacy.merge(df_pdf,on="txt_fn",how="left")
df

,doc_path,num_pages,metadata,is_empty,txt_file,source_fn,txt_fn,collection,error,exception_text,num_chars,num_tokens,num_sentences,langs_raw,langs_small
0,/home/sh/hsa/papers/dataset/extracted/Final_te...,19.0,"{'format': 'PDF 1.4', 'title': 'EA', 'author':...",False,/tmp/FULL_CLI/Final_terms/DE000HLB7C13.pdf.txt,Final_terms/Country_of_origin_DE/lang_DE_EN/DE...,Final_terms/DE000HLB7C13.pdf.txt,Final_terms,NaN,NaN,54319.0,8365.0,340.0,"Counter({'GERMAN': 0.9822987680899414, 'PORTUG...","{'GERMAN': 0.9873767732628036, 'PORTUGUESE': 0..."
1,/home/sh/hsa/papers/dataset/extracted/Final_te...,14.0,"{'format': 'PDF 1.6', 'title': '', 'author': '...",False,/tmp/FULL_CLI/Final_terms/DE000NWB2ME7.pdf.txt,Final_terms/Country_of_origin_DE/lang_DE_EN/DE...,Final_terms/DE000NWB2ME7.pdf.txt,Final_terms,NaN,NaN,23605.0,4354.0,183.0,"Counter({'ENGLISH': 0.45957740009186954, 'GERM...","{'ENGLISH': 0.47802197802197804, 'GERMAN': 0.4..."
2,/home/sh/hsa/papers/dataset/extracted/Final_te...,25.0,"{'format': 'PDF 1.5', 'title': '', 'author': '...",False,/tmp/FULL_CLI/Final_terms/DE000DFK0TS8.pdf.txt,Final_terms/Country_of_origin_DE/lang_DE_EN/DE...,Final_terms/DE000DFK0TS8.pdf.txt,Final_terms,NaN,NaN,70180.0,12733.0,748.0,"Counter({'GERMAN': 0.4806122448979592, 'ENGLIS...","{'GERMAN': 0.4943883730318934, 'ENGLISH': 0.46..."
3,/home/sh/hsa/papers/dataset/extracted/Final_te...,15.0,"{'format': 'PDF 1.4', 'title': 'Dokumentation'...",False,/tmp/FULL_CLI/Final_terms/DE000LB2CZ54.pdf.txt,Final_terms/Country_of_origin_DE/lang_DE_EN/DE...,Final_terms/DE000LB2CZ54.pdf.txt,Final_terms,NaN,NaN,46594.0,7544.0,291.0,"Counter({'GERMAN': 0.976004242343895, 'ENGLISH...","{'GERMAN': 0.97990150405963, 'ENGLISH': 0.0107..."
4,/home/sh/hsa/papers/dataset/extracted/Final_te...,25.0,"{'format': 'PDF 1.5', 'title': '', 'author': '...",False,/tmp/FULL_CLI/Final_terms/DE000BLB80W4.pdf.txt,Final_terms/Country_of_origin_DE/lang_DE_EN/DE...,Final_terms/DE000BLB80W4.pdf.txt,Final_terms,NaN,NaN,59569.0,9578.0,350.0,"Counter({'GERMAN': 0.9730660820544942, 'ENGLIS...","{'GERMAN': 0.9809513786571248, 'ENGLISH': 0.00..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13656,/home/sh/hsa/papers/dataset/extracted/Final_te...,3.0,"{'format': 'PDF 1.4', 'title': '', 'author': '...",True,NaN,Final_terms/Country_of_origin_DE/lang_DE_EN/DE...,Final_terms/DE000A32VMZ6.pdf.txt,Final_terms,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13657,/home/sh/hsa/papers/dataset/extracted/Final_te...,41.0,"{'format': 'PDF 1.5', 'title': '', 'author': '...",False,/tmp/FULL_CLI/Final_terms/DE000DDA0SE4.pdf.txt,Final_terms/Country_of_origin_DE/lang_DE_EN/DE...,Final_terms/DE000DDA0SE4.pdf.txt,Final_terms,NaN,NaN,103173.0,19253.0,837.0,"Counter({'GERMAN': 0.4852009554470869, 'ENGLIS...","{'GERMAN': 0.4989853679376268, 'ENGLISH': 0.46..."
13658,/home/sh/hsa/papers/dataset/extracted/Monthly_...,156.0,"{'format': 'PDF 1.6', 'title': 'Monatsbericht ...",False,/tmp/FULL_CLI/Monthly_reports_BB/2012-08-monat...,Monthly_reports_BB/2012-08-monatsbericht-data.pdf,Monthly_reports_BB/2012-08-monatsbericht-data....,Monthly_reports_BB,NaN,NaN,676991.0,192860.0,3703.0,"Counter({'GERMAN': 0.5177500220352252, 'DUTCH'...","{'GERMAN': 0.5275085180000528, 'DUTCH': 0.3572..."
13659,/home/sh/hsa/papers/dataset/extracted/Final_te...,10.0,"{'format': 'PDF 1.5', 'title': '1866', 'author...",False,/tmp/FULL_CLI/Final_terms/DE000MHB4255.pdf.txt,Final_terms/Country_of_origin_DE/lang_DE_EN/DE...,Final_terms/DE000MHB4255.pdf.txt,Final_terms,NaN,NaN,18636.0,3483.0,205.0,"Counter({'GERMAN': 0.5014492753623189, 'ENGLIS...","{'GERMAN': 0.5112293144208038, 'ENGLISH': 0.45..."


In [56]:
df_clean = df[(df.error.isna()) & (df.is_empty == False)]
# drop the now-uneeeded columns with error data
df_clean.drop(columns=["error", "exception_text","is_empty"], inplace=True)
df_clean.head()

/tmp/ipykernel_605118/4061407583.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(columns=["error", "exception_text","is_empty"], inplace=True)


,doc_path,num_pages,metadata,txt_file,source_fn,txt_fn,collection,num_chars,num_tokens,num_sentences,langs_raw,langs_small
0,/home/sh/hsa/papers/dataset/extracted/Final_te...,19.0,"{'format': 'PDF 1.4', 'title': 'EA', 'author':...",/tmp/FULL_CLI/Final_terms/DE000HLB7C13.pdf.txt,Final_terms/Country_of_origin_DE/lang_DE_EN/DE...,Final_terms/DE000HLB7C13.pdf.txt,Final_terms,54319.0,8365.0,340.0,"Counter({'GERMAN': 0.9822987680899414, 'PORTUG...","{'GERMAN': 0.9873767732628036, 'PORTUGUESE': 0..."
1,/home/sh/hsa/papers/dataset/extracted/Final_te...,14.0,"{'format': 'PDF 1.6', 'title': '', 'author': '...",/tmp/FULL_CLI/Final_terms/DE000NWB2ME7.pdf.txt,Final_terms/Country_of_origin_DE/lang_DE_EN/DE...,Final_terms/DE000NWB2ME7.pdf.txt,Final_terms,23605.0,4354.0,183.0,"Counter({'ENGLISH': 0.45957740009186954, 'GERM...","{'ENGLISH': 0.47802197802197804, 'GERMAN': 0.4..."
2,/home/sh/hsa/papers/dataset/extracted/Final_te...,25.0,"{'format': 'PDF 1.5', 'title': '', 'author': '...",/tmp/FULL_CLI/Final_terms/DE000DFK0TS8.pdf.txt,Final_terms/Country_of_origin_DE/lang_DE_EN/DE...,Final_terms/DE000DFK0TS8.pdf.txt,Final_terms,70180.0,12733.0,748.0,"Counter({'GERMAN': 0.4806122448979592, 'ENGLIS...","{'GERMAN': 0.4943883730318934, 'ENGLISH': 0.46..."
3,/home/sh/hsa/papers/dataset/extracted/Final_te...,15.0,"{'format': 'PDF 1.4', 'title': 'Dokumentation'...",/tmp/FULL_CLI/Final_terms/DE000LB2CZ54.pdf.txt,Final_terms/Country_of_origin_DE/lang_DE_EN/DE...,Final_terms/DE000LB2CZ54.pdf.txt,Final_terms,46594.0,7544.0,291.0,"Counter({'GERMAN': 0.976004242343895, 'ENGLISH...","{'GERMAN': 0.97990150405963, 'ENGLISH': 0.0107..."
4,/home/sh/hsa/papers/dataset/extracted/Final_te...,25.0,"{'format': 'PDF 1.5', 'title': '', 'author': '...",/tmp/FULL_CLI/Final_terms/DE000BLB80W4.pdf.txt,Final_terms/Country_of_origin_DE/lang_DE_EN/DE...,Final_terms/DE000BLB80W4.pdf.txt,Final_terms,59569.0,9578.0,350.0,"Counter({'GERMAN': 0.9730660820544942, 'ENGLIS...","{'GERMAN': 0.9809513786571248, 'ENGLISH': 0.00..."


In [57]:
# Check that the BB monthly reports without corresponding spacy metadata are still there, named `ocr_...`
#df_clean[(df_clean.collection=="Monthly_reports_BB")].txt_fn
df_clean[(df_clean.collection=="Monthly_reports_BB") & (df_clean.num_sentences.isna())]

,doc_path,num_pages,metadata,txt_file,source_fn,txt_fn,collection,num_chars,num_tokens,num_sentences,langs_raw,langs_small


In [58]:
# Entire ds, including the files without text
TARGET_PATH = Path("../data/merge_res_complete.csv")
# ds only with files containing text
TARGET_PATH_CLEAN = Path("../data/merge_res.csv")

df.to_csv(TARGET_PATH,index=False)
df_clean.to_csv(TARGET_PATH_CLEAN,index=False)
print(TARGET_PATH.resolve())

/home/sh/hsa/papers/german-financial-dataset/data/merge_res_complete.csv
